In [1]:
from utils import copy_csv_results, extract_summary_stats
from pathlib import Path
import os
import pandas as pd

In [2]:
# Define your output directory (folder containing the results of the analysis in 1_image_analysis)
PARENT_FOLDER = Path("./output")

# Define your username (subfolder name under the output directory)
USERNAME = "Andrew"

# Merge parent_folder and username into a Path
results_directory = PARENT_FOLDER / USERNAME

In [3]:
# Copies .csv files containing per organoid results into a per_organoid_stats folder within results_directory
copy_csv_results(results_directory)

In [4]:
# Define the directory containing the copied per_organoid_stats
csv_results_path = Path(os.path.join(results_directory, "per_organoid_stats"))

# Initialize an empty DataFrame to collect all summary dataframes
final_df = pd.DataFrame()

# Scan for .csv files in each subdirectory
for csv_path in csv_results_path.glob("*.csv"):
    summary_df = extract_summary_stats(csv_path)
    # Append the summary_df to the final_df
    final_df = pd.concat([final_df, summary_df], ignore_index=True)
    
# Create the summary_stats directory if it doesn't exist
summary_stats_directory = Path(results_directory, "summary_stats")
summary_stats_directory.mkdir(parents=True, exist_ok=True)

# Save the final_df as a .csv file under the summary_stats directory
final_csv_path = summary_stats_directory / "summary_stats.csv"
final_df.to_csv(final_csv_path, index=False)